This module handles all data preprocessing tasks including:
- Loading training and test datasets
- Data cleaning and missing value handling
- Feature engineering
- Categorical encoding
- Data validation and quality checks

Date: 2025/07/15

### Xử lý và chuẩn bị dữ liệu cho mô hình Conformal Prediction

#### Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
import warnings
import os
warnings.filterwarnings('ignore')

#### Đọc dữ liệu từ file CSV

In [2]:
print("Đang đọc dữ liệu...")
dataset_df = pd.read_csv('dataset/dataset.csv')
test_df = pd.read_csv('dataset/test.csv')

print(f"Kích thước dataset: {dataset_df.shape}")
print(f"Kích thước test: {test_df.shape}")

Đang đọc dữ liệu...
Kích thước dataset: (200000, 47)
Kích thước test: (200000, 46)


#### Kiểm tra và xử lý dữ liệu thiếu

In [3]:
print("\nThông tin về dữ liệu thiếu trong dataset:")
missing_dataset = dataset_df.isnull().sum()
print(missing_dataset[missing_dataset > 0])

print("\nThông tin về dữ liệu thiếu trong test:")
missing_test = test_df.isnull().sum()
print(missing_test[missing_test > 0])


Thông tin về dữ liệu thiếu trong dataset:
sale_nbr       42182
subdivision    17550
submarket       1717
dtype: int64

Thông tin về dữ liệu thiếu trong test:
sale_nbr       42412
subdivision    17550
submarket       1718
dtype: int64


#### Xử lý cột ngày tháng

In [4]:
def extract_date_features(df, date_column='sale_date'):
    """
    Trích xuất các đặc trưng từ cột ngày tháng
    """
    df_copy = df.copy()

    if date_column in df_copy.columns:
        df_copy[date_column] = pd.to_datetime(df_copy[date_column])

        df_copy['sale_year'] = df_copy[date_column].dt.year
        df_copy['sale_month'] = df_copy[date_column].dt.month
        df_copy['sale_quarter'] = df_copy[date_column].dt.quarter
        df_copy['sale_day_of_week'] = df_copy[date_column].dt.dayofweek
        df_copy['sale_day_of_month'] = df_copy[date_column].dt.day

        df_copy = df_copy.drop(columns=[date_column])

        print(f"Đã trích xuất các đặc trưng từ {date_column}")

    return df_copy

dataset_with_date_features = extract_date_features(dataset_df, 'sale_date')
test_with_date_features = extract_date_features(test_df, 'sale_date')

Đã trích xuất các đặc trưng từ sale_date
Đã trích xuất các đặc trưng từ sale_date


#### Xử lý các cột categorical

In [5]:
categorical_columns = ['sale_warning', 'join_status', 'city', 'zoning', 'subdivision', 'submarket']

def preprocess_categorical(df, categorical_cols, label_encoders=None, is_training=True):
    df_processed = df.copy()

    if is_training:
        encoders = {}
        for col in categorical_cols:
            if col in df_processed.columns:
                le = LabelEncoder()
                df_processed[col] = df_processed[col].fillna('unknown')
                df_processed[col] = df_processed[col].astype(str)

                unique_values = df_processed[col].unique().tolist()
                if 'unknown' not in unique_values:
                    unique_values.append('unknown')

                le.fit(unique_values)
                df_processed[col] = le.transform(df_processed[col])
                encoders[col] = le
        return df_processed, encoders
    else:
        for col in categorical_cols:
            if col in df_processed.columns and col in label_encoders:
                df_processed[col] = df_processed[col].fillna('unknown')
                df_processed[col] = df_processed[col].astype(str)

                le = label_encoders[col]

                unknown_mask = ~df_processed[col].isin(le.classes_)
                df_processed.loc[unknown_mask, col] = 'unknown'

                df_processed[col] = le.transform(df_processed[col])
        return df_processed

dataset_processed, label_encoders = preprocess_categorical(dataset_with_date_features, categorical_columns, is_training=True)

#### Xử lý dữ liệu thiếu cho các cột số

In [6]:
numeric_columns = dataset_processed.select_dtypes(include=[np.number]).columns.tolist()
if 'sale_price' in numeric_columns:
    numeric_columns.remove('sale_price')

for col in numeric_columns:
    if dataset_processed[col].isnull().sum() > 0:
        median_val = dataset_processed[col].median()
        dataset_processed[col] = dataset_processed[col].fillna(median_val)
        print(f"Điền {col} với giá trị median: {median_val}")

Điền sale_nbr với giá trị median: 2.0


#### Tách features và target

In [7]:
feature_columns = [col for col in dataset_processed.columns if col not in ['sale_price']]
X = dataset_processed[feature_columns]
y = dataset_processed['sale_price']

print(f"\nSố lượng features: {X.shape[1]}")
print(f"Các đặc trưng từ ngày: sale_year, sale_month, sale_quarter, sale_day_of_week, sale_day_of_month")
print(f"Tên các features: {X.columns.tolist()}...")


Số lượng features: 50
Các đặc trưng từ ngày: sale_year, sale_month, sale_quarter, sale_day_of_week, sale_day_of_month
Tên các features: ['id', 'sale_nbr', 'sale_warning', 'join_status', 'join_year', 'latitude', 'longitude', 'area', 'city', 'zoning', 'subdivision', 'present_use', 'land_val', 'imp_val', 'year_built', 'year_reno', 'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt', 'grade', 'fbsmt_grade', 'condition', 'stories', 'beds', 'bath_full', 'bath_3qtr', 'bath_half', 'garb_sqft', 'gara_sqft', 'wfnt', 'golf', 'greenbelt', 'noise_traffic', 'view_rainier', 'view_olympics', 'view_cascades', 'view_territorial', 'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm', 'view_otherwater', 'view_other', 'submarket', 'sale_year', 'sale_month', 'sale_quarter', 'sale_day_of_week', 'sale_day_of_month']...


#### Chia tập train và validation

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

print(f"\nKích thước tập train: {X_train.shape}")
print(f"Kích thước tập validation: {X_val.shape}")


Kích thước tập train: (160000, 50)
Kích thước tập validation: (40000, 50)


#### Chuẩn hóa dữ liệu

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print("\nĐã hoàn thành chuẩn hóa dữ liệu")


Đã hoàn thành chuẩn hóa dữ liệu


#### Xử lý tập test

In [10]:
test_processed = preprocess_categorical(test_with_date_features, categorical_columns, label_encoders, is_training=False)

for col in feature_columns:
    if col in test_processed.columns and test_processed[col].dtype in ['int64', 'float64'] and test_processed[col].isnull().sum() > 0:
        median_val = dataset_processed[col].median()
        test_processed[col] = test_processed[col].fillna(median_val)

X_test = test_processed[feature_columns]
X_test_scaled = scaler.transform(X_test)

print(f"Kích thước tập test sau xử lý: {X_test_scaled.shape}")
print(f"Đã xử lý tập test với {len(feature_columns)} features")

Kích thước tập test sau xử lý: (200000, 50)
Đã xử lý tập test với 50 features


#### Lưu dữ liệu đã xử lý

In [11]:
os.makedirs('processed', exist_ok=True)

joblib.dump(scaler, 'processed/scaler.pkl')
joblib.dump(label_encoders, 'processed/label_encoders.pkl')
joblib.dump(feature_columns, 'processed/feature_columns.pkl')

np.save('processed/X_train_scaled.npy', X_train_scaled)
np.save('processed/X_val_scaled.npy', X_val_scaled)
np.save('processed/X_test_scaled.npy', X_test_scaled)
np.save('processed/y_train.npy', y_train.values)
np.save('processed/y_val.npy', y_val.values)

print("\nĐã lưu tất cả dữ liệu đã xử lý:")
print("- processed/scaler.pkl")
print("- processed/label_encoders.pkl")
print("- processed/feature_columns.pkl")
print("- processed/X_train_scaled.npy")
print("- processed/X_val_scaled.npy")
print("- processed/X_test_scaled.npy")
print("- processed/y_train.npy")
print("- processed/y_val.npy")


Đã lưu tất cả dữ liệu đã xử lý:
- processed/scaler.pkl
- processed/label_encoders.pkl
- processed/feature_columns.pkl
- processed/X_train_scaled.npy
- processed/X_val_scaled.npy
- processed/X_test_scaled.npy
- processed/y_train.npy
- processed/y_val.npy


#### Thống kê cơ bản về target variable

In [12]:
print(f"\nThống kê về sale_price:")
print(f"Mean: {y.mean():,.0f}")
print(f"Median: {y.median():,.0f}")
print(f"Std: {y.std():,.0f}")
print(f"Min: {y.min():,.0f}")
print(f"Max: {y.max():,.0f}")

print("\nHoàn thành xử lý dữ liệu!")


Thống kê về sale_price:
Mean: 584,149
Median: 459,950
Std: 417,060
Min: 50,293
Max: 2,999,950

Hoàn thành xử lý dữ liệu!
